## This script can be found on [GitHub](https://github.com/Basem-Qahtan/STEM-data-processing-scripts) along with other scripts for STEM data processing algorithms.

##### --------------------------------------------------------------------------------------------------------------------------------------------
# STEM-Cathodlumensence datasets  

Cathodlumensence (CL) data aquisition in TEM is used in applications such as LEDs to collect information on the wavelength of the emitted photons from quantum wells for example. in some cases, the sample is the electron beam with a short dewll time to avoid sample's movement. However, this will result in a lo signal to noise ratio (SNR). This notebook increases the SNR of a noisy CL datasets through the Principal component analysis (PCA) denoising.


## Author

* 15/12/2024 Basem Qahtan - Developed as part of a Ph.D. research at the Italian Institue of Technology (iit)/ University of Genoa (uniGe) in italy and KAUST university in KSA.
## Changes

* 15/10/2024 Update: Added auto cropping and rebinning of EDX map prior to the principal component analysis (PCA) denoising.
* 10/11/2024 Update: Removal of negative signals and high intense signals at rnadom pixels (artifacts). 
* 30/06/2025 Update: Added the option to use spikes_diagnosis() function by lumispy to remove signal spikes if needed. 


## Requirements

* HyperSpy 1.7.4 or higher

## <a id='top'></a> Contents

1. <a href='#dat'> Specimen & Data</a>
2. <a href='#load'> Load the CL data</a>
3. <a href='#crop'> Crop negative values and cap the intensity of pixels with extreme intensity (if needed)</a>
4. <a href='#pca'> Apply PCA decomposition on the CL map </a>
5. <a href='#save'> Save the denoised CL dataset in hspy format</a>


# <a id='dat'></a> 1. Specimen & Data

Samples used in this example were test data as shown i figure 1 below with high noise (left)

<img src="images/CL data PCA denoising.png" style="height:400px;">
Figure 1: Reconstructed Cathodlumenscence (CL) data after PCA denoising resulting in an enhanced Signal to noise ratio (SNR).


## Import required libraries

In [1]:
%matplotlib qt
import hyperspy.api as hs
import numpy as np
import scipy.misc
import scipy
import matplotlib.pyplot as plt
import os
import re


# <a id='load'></a> 2. Load the CL data


In [16]:
# Specify the directory path
directory = os.getcwd()

# Get a list of files in the directory
file_list = os.listdir(directory)

# Filter files with the .emd or .bcf extension, you may add your file extension here if it is different.
emd_files = [file for file in file_list if file.endswith(".emd") or file.endswith(".bcf") or file.endswith(".dm4")]

def extract_number(filename):
    # Extract the number from the filename using regex
    match = re.search(r'\((\d+)\)', filename)
    if match:
        return int(match.group(1))
    return 0  # Return 0 if no number is found

# Sort the list using the custom key function
sorted_emd_files = sorted(emd_files, key=extract_number)

intitial_k = []
dynamic_K = []

# Load and process the .emd files using Hyperspy
for file in sorted_emd_files:
    # Construct the full file path
    file_path = os.path.join(directory, file)
    
    # Load the file using Hyperspy
    SI = hs.load(file_path)  

    # Split the string based on the '.' delimiter
    name = file.split('.')[0]

    CL_found = False
    HAADF = None
    EDS = None
    CL = None

    # Check if SI is a list (subscriptable) or a single signal
    if isinstance(SI, list):
        for i, signal in enumerate(SI):
            if signal.metadata.General.title == 'HAADF':
                HAADF = signal
                HAADFindex = i
            elif signal.metadata.General.title in ("EDS", "EDX"):
                if signal.data.ndim == 3:  # Ensure it has three dimensions
                    EDS = signal
                    EDSindex = i
            elif signal.metadata.General.title == 'CLSpectrum' or signal.metadata.Signal.signal_type=="CL":
                CL = signal
                CL_found = True
    else:
        # SI is a single signal
        if SI.metadata.General.title == 'HAADF':
            HAADF = SI
        elif SI.metadata.General.title in ("EDS", "EDX"):
            if SI.data.ndim == 3:
                EDS = SI
        elif SI.metadata.General.title == 'CLSpectrum' or SI.metadata.Signal.signal_type=="CL":
            CL = SI
            CL_found = True

    if CL_found == False: 
        print("CL map is not found in the dataset ")

    # Print the extracted values
    print("Sample name:", name)
  #  print("HAADF:", "Found" if HAADF is not None else "Not found")
  #  print("EDX:", "Found" if EDS is not None else "Not found")
    print("CL map:", "Found" if CL is not None else "Not found in the dataset (Cant continue the processing of CL data using this script)")
    print("------------------------------------------------------")


Sample name: PCA_TestData_03_RedBlue_5kV_HighNoise
CL map: Found
------------------------------------------------------


In [17]:
# check the varibale CL contains the SI dataset
CL.metadata

├── Acquisition_instrument
│   ├── Detector
│   │   ├── binning = (8, 400)
│   │   ├── detector_type = CCD
│   │   ├── exposure_per_frame = 0.009999999776482582
│   │   ├── frames = 2
│   │   ├── integration_time = 0.019999999552965164
│   │   ├── pixel_size = 20.0
│   │   ├── pmt_voltage = 1000.0
│   │   ├── processing = Dark Subtracted
│   │   ├── saturation_fraction <list>
│   │   │   ╠══ [0] = 0.001152691780589521
│   │   │   ╠══ [1] = 0.0011946546146646142
│   │   │   ╠══ [10] = 0.0010058237239718437
│   │   │   ╠══ [100] = 0.0011832098243758082
│   │   │   ╠══ [1000] = 0.0012080057058483362
│   │   │   ╠══ [1001] = 0.0010636810911819339
│   │   │   ╠══ [1002] = 0.0008882023976184428
│   │   │   ╠══ [1003] = 0.0008920171530917287
│   │   │   ╠══ [1004] = 0.0010001015616580844
│   │   │   ╠══ [1005] = 0.0010337987914681435
│   │   │   ╠══ [1006] = 0.0009657688206061721
│   │   │   ╠══ [1007] = 0.001009638886898756
│   │   │   ╠══ [1008] = 0.0009708552970550954
│   │   │   ╠══ [1009] = 0.0009511458920314908
│   │   │   ╠══ [101] = 0.0008869306766428053
│   │   │   ╠══ [1010] = 0.0010363422334194183
│   │   │   ╠══ [1011] = 0.0010802114848047495
│   │   │   ╠══ [1012] = 0.0009377937531098723
│   │   │   ╠══ [1013] = 0.0010554156033322215
│   │   │   ╠══ [1014] = 0.0011971971252933145
│   │   │   ╠══ [1015] = 0.0013141832314431667
│   │   │   ╠══ [1016] = 0.0013415226712822914
│   │   │   ╠══ [1017] = 0.0015017424011602998
│   │   │   ╠══ [1018] = 0.0017058312660083175
│   │   │   ╠══ [1019] = 0.0016619616653770208
│   │   │   ╠══ [102] = 0.001173672964796424
│   │   │   ╠══ [1020] = 0.0018870321800932288
│   │   │   ╠══ [1021] = 0.0024961214512586594
│   │   │   ╠══ [1022] = 0.0022780445870012045
│   │   │   ╠══ [1023] = 0.002461153082549572
│   │   │   ╠══ [1024] = 0.0028318199329078197
│   │   │   ╠══ [1025] = 0.0027129268273711205
│   │   │   ╠══ [1026] = 0.002738358685746789
│   │   │   ╠══ [1027] = 0.0024121971800923347
│   │   │   ╠══ [1028] = 0.002304747933521867
│   │   │   ╠══ [1029] = 0.0019913024734705687
│   │   │   ╠══ [103] = 0.001014089328236878
│   │   │   ╠══ [1030] = 0.0019200937822461128
│   │   │   ╠══ [1031] = 0.001750336610712111
│   │   │   ╠══ [1032] = 0.0014445210108533502
│   │   │   ╠══ [1033] = 0.0013739480637013912
│   │   │   ╠══ [1034] = 0.0011539636179804802
│   │   │   ╠══ [1035] = 0.0009918364230543375
│   │   │   ╠══ [1036] = 0.0010585946729406714
│   │   │   ╠══ [1037] = 0.0009702191455289721
│   │   │   ╠══ [1038] = 0.0009829356567934155
│   │   │   ╠══ [1039] = 0.0010039168410003185
│   │   │   ╠══ [104] = 0.0010941986693069339
│   │   │   ╠══ [1040] = 0.0010662241838872433
│   │   │   ╠══ [1041] = 0.0010471503483131528
│   │   │   ╠══ [1042] = 0.001052236882969737
│   │   │   ╠══ [1043] = 0.00083479582099244
│   │   │   ╠══ [1044] = 0.0011724017094820738
│   │   │   ╠══ [1045] = 0.0011768520344048738
│   │   │   ╠══ [1046] = 0.0010865697404369712
│   │   │   ╠══ [1047] = 0.0010306196054443717
│   │   │   ╠══ [1048] = 0.0011323466897010803
│   │   │   ╠══ [1049] = 0.0009676766349002719
│   │   │   ╠══ [105] = 0.0010083671659231186
│   │   │   ╠══ [1050] = 0.000973398273345083
│   │   │   ╠══ [1051] = 0.001210549264214933
│   │   │   ╠══ [1052] = 0.0011367971310392022
│   │   │   ╠══ [1053] = 0.0009664048557169735
│   │   │   ╠══ [1054] = 0.0010636807419359684
│   │   │   ╠══ [1055] = 0.001328170532360673
│   │   │   ╠══ [1056] = 0.0010026446543633938
│   │   │   ╠══ [1057] = 0.0009505097405053675
│   │   │   ╠══ [1058] = 0.0012010122882202268
│   │   │   ╠══ [1059] = 0.0010318913264200091
│   │   │   ╠══ [106] = 0.00127031363081187
│   │   │   ╠══ [1060] = 0.0010427000233903527
│   │   │   ╠══ [1061] = 0.0009854781674221158
│   │   │   ╠══ [1062] = 0.0011469696182757616
│   │   │   ╠══ [1063] = 0.0010172679321840405
│   │   │   ╠══ [1064] = 0.0009308003354817629
│   │   │   ╠══ [1065] = 0.0011514200596138835
│   │   │   ╠══ [1066] = 0.0010700388811528683
│   │   │   ╠══ [1067] = 0.001607

#### When plotting the signal below: if you cant use the keyboard arrows to navigate through the signal or navigation axes, try to drag the small red square (shown in figure 2 below in the top left corner of the CL map) by holding the right click on the mouse 

In [18]:
CL.plot()

# <a id='crop'></a> 3. Crop negative values and remove any spikes (if needed)

<img src="images/high intensity capping in CL data.png" style="height:400px;">
Figure 2: Cropping wavelengths counts with extremly high intensities (due to artifacts) as it will affect the components in PCA.


## 2 options can be selected: 
### 1- Spikes_diagnosis() function by lumispy 
### 2- Average of the top 30 intensities

## User can switch between the 2 methods to assess the result (original signal will not be affected until moving to the next step)
#### --------------------------------------------------------------------------------------------------------------------------------------------

# 1- Rmoving spikes using spikes_diagnosis() function by lumispy in HyperSpy

In [26]:
CL_copy=CL.deepcopy()
# Step 1: Run diagnosis to determine threshold
CL_copy.spikes_diagnosis()  # Check the derivative histogram's first zero-crossing (e.g., 10)

# Step 2: Remove spikes using the observed threshold
CL_copy.spikes_removal_tool(threshold=200, interactive=False)
CL_data_clipped=CL_copy
CL_data_clipped.plot()

# 2- Rmoving spikes using the average of the top 30 Intensities
### If the spikes_diagnosis() function above didnt remove the spikes in the CL dataset, try capping the max signal intiensity based on the average of the top 30 conunts:

In [20]:
CL_copy=CL.deepcopy()
# Assuming you've already calculated max_intensities
max_intensities = np.max(CL_copy.data, axis=2)

# Get the indices of the top 50 intensity values
top_30_indices = np.argpartition(max_intensities.flatten(), -30)[-30:]

# Get the actual top 50 intensity values
top_30_intensities = max_intensities.flatten()[top_30_indices]

# Sort the top 50 intensities in descending order
top_30_intensities_sorted = np.sort(top_30_intensities)[::-1]

In [21]:
# check the highest intensity counts in the top 30 pixels
top_30_intensities_sorted

array([14050.088  ,  2956.1416 ,   388.51495,   366.3483 ,   364.37494,
         342.3483 ,   334.0616 ,   321.13828,   319.94827,   318.76495,
         315.18164,   312.97836,   311.76495,   307.18158,   306.5616 ,
         305.68158,   305.30496,   300.93164,   295.59827,   294.43158,
         293.8483 ,   289.84824,   283.13828,   281.395  ,   281.3483 ,
         279.05496,   278.9316 ,   277.76495,   273.0983 ,   272.72165],
      dtype=float32)

In [22]:
# cap the max intensity across the CL map based on the mean value below:
top_30_intensities_sorted.mean()

854.2831

In [25]:
CL_data_clipped = np.clip(CL, a_min= None,a_max=top_30_intensities_sorted.mean())#None   #top_50_intensities_sorted.mean()
CL_data_clipped.plot()

# Removing negative axis values in the CL datasets (if needed)
###  Negative values in CL datasets are due to background subtraction, it is better to keep these values but if needed they can be removed to avoid their effect on the PCA denoising output

In [5]:
# Assuming 'cl_data' is your 3D CL dataset
CL_data_clipped = np.clip(CL, a_min=0, a_max=None) #None   #top_50_intensities_sorted.mean()

In [6]:
#check the CL map after cropping negative / high intensity counts:
CL_data_clipped.plot()

# <a id='pca'></a> 4. Apply PCA decomposition on the CL map

<img src="images/PCA on CL data.png" style="height:450px;">
Figure 3: Cropping wavelengths counts with extremly high intensities (due to artifacts) as it will affect the components in PCA.


In [7]:
CL_data_clipped.change_dtype("float32")

CL_data_clipped.decomposition()

Decomposition info:
  normalize_poissonian_noise=False
  algorithm=SVD
  output_dimension=None
  centre=None


## Plot the scree plot and the individual components:

#### After plotting the decomposition results below: if you cant navigate between the components using the keyboard arrows, use the slider bar opened in 3rd smal window or in the notebook itself as shown in figure 4 below.

In [8]:
CL_data_clipped.plot_decomposition_results()
CL_data_clipped.plot_explained_variance_ratio()

<Axes: title={'center': 'PCA_TestData_03_RedBlue_5kV_HighNoise\nPCA Scree Plot'}, xlabel='Principal component index', ylabel='Proportion of variance'>

<img src="images/slider bar.png" style="height:450px;">
Figure 4: Alternative option to cycle beween the PCA components if keyboard arrows or bindings doesnt work

## Select the number of components to be used in reconstructing the CL map:

#### If you want to use the first 4 components you can type:
###### CLdata =cl_data_clipped.get_decomposition_model(4)
##### --------------------------------------------------------------------------------------------------
#### If you want to use the 1st, 2nd and 4th components you can type:
###### CLdata =cl_data_clipped.get_decomposition_model([0,1,3])   >>>> numbering starts from 0

In [9]:
CLdata=CL_data_clipped.get_decomposition_model(2)

In [10]:
CLdata

<CLSpectrum, title: PCA_TestData_03_RedBlue_5kV_HighNoise model from decomposition with 2 components, dimensions: (50, 50|167)>

In [11]:
CLdata.plot()

In [12]:
CLdata

<CLSpectrum, title: PCA_TestData_03_RedBlue_5kV_HighNoise model from decomposition with 2 components, dimensions: (50, 50|167)>

In [29]:
CL2=CLdata.inav[32,47]
CL2

<CLSpectrum, title: PCA_TestData_03_RedBlue_5kV_HighNoise model from decomposition with 2 components, dimensions: (|167)>

In [31]:
CL1=CLdata.inav[25,22]
CL1


<CLSpectrum, title: PCA_TestData_03_RedBlue_5kV_HighNoise model from decomposition with 2 components, dimensions: (|167)>

In [39]:
import matplotlib.pyplot as plt

# Assuming CL1 and CL2 are HyperSpy signals with the same signal dimension (e.g., spectra)

# Extract the signal data arrays (intensity vs channel/wavelength)
x1 = CL1.axes_manager.signal_axes[0].axis  # e.g., wavelength or energy axis for CL1
y1 = CL1.data

x2 = CL2.axes_manager.signal_axes[0].axis  # axis for CL2
y2 = CL2.data

plt.figure(figsize=(8,5))
plt.plot(x1, y1, label='CL1', color='blue')
plt.plot(x2, y2, label='CL2', color='red')

plt.xlabel("Wavelength axis (nm)", fontsize=16)  # e.g., 'Wavelength (nm)'
plt.ylabel('Intensity (Counts)', fontsize=16)
plt.title('Comparison of CL1 and CL2 Spectra')
plt.legend()
plt.grid(True)
plt.show()


In [36]:
CL1.axes_manager.signal_axes.name

AttributeError: 'tuple' object has no attribute 'name'

# <a id='save'></a> 5. Save the denoised CL dataset in hspy format


In [ ]:
CLdata.save(name+"  (PCA denoised).hspy")
